# 4章 地理空間データ分析のクラウド化：Google Earth Engineとその他のツール

## 4.1 Google Earth Engineの設定

## 4.2 GEEコンソールとgeemapの使用法

## 4.3 geemapを利用する

In [1]:
import os
import ee
import geemap

# カーソルを再起動する時に実行
# geemap.update_package()

In [2]:
Map = geemap.Map(center=(40.0, -100.0), zoom=4)
Map.basemap()
Map

Map(center=[40.0, -100.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

## 4.4 Landsat9の画像コレクションを探索する

In [3]:
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
print(f"LANDSAT/LC09/C02/T1_L2 collections: {collection.size().getInfo():,}")

LANDSAT/LC09/C02/T1_L2 collections: 528,268


In [4]:
median = collection.median()

## 4.5 スペクトルバンドを使った作業

In [5]:
# すべての画像から合成画像を作成する
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [6]:
dataset = apply_scale_factors(median)

In [7]:
# 
vis_natural = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_nir = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [8]:
Map.addLayer(dataset, vis_natural, 'True color (432)')
Map.addLayer(dataset, vis_nir, 'True color (543)')
Map

Map(center=[40.0, -100.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [9]:
vis_params = [
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3},
    {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0.0, 'max': 0.3},
    {'bands': ['SR_B7', 'SR_B6', 'SR_B4'], 'min': 0.0, 'max': 0.3},
    {'bands': ['SR_B6', 'SR_B5', 'SR_B2'], 'min': 0.0, 'max': 0.3},
]

labels = [
    'Natural Color (4, 3, 2)',
    'Natural Color (5, 4, 3)',
    'Natural Color (7, 6, 4)',
    'Natural Color (6, 5, 2)',
]

geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[-3.4653, -62.2159],
    zoom=4,
    ee_objects=[dataset],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

## 4.6 国土被覆データベースのベースマップ

### 4.6.1 データへのアクセス

In [10]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [11]:
# geemapにてJavaScript→Python変換
# The code has been copied to the clipboard. 
# Press Ctrl+V to in a code cell to paste it.
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# The collection contains images for multiple years and regions in the USA.
# print('"Products":', dataset.aggregate_array('"system":index').getInfo())
print(f'"Products": {dataset.aggregate_array("system:index").getInfo()}')

# Filter the collection to the 2016 product.
# nlcd2016 = dataset.filter(ee.Filter.eq('"system":index', '2016')).first()
# 修正
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first()

# Each product has multiple bands for describing aspects of land cover.
print(f'"Bands": {nlcd2016.bandNames().getInfo()}')

# Select the land cover band.
landcover = nlcd2016.select('landcover')

# Display land cover on the map.
Map.setCenter(-95, 38, 5)
Map.addLayer(landcover, None, 'Landcover')


"Products": ['2001', '2004', '2006', '2008', '2011', '2013', '2016', '2019']
"Bands": ['landcover', 'impervious', 'impervious_descriptor']


In [12]:
legends = geemap.legends.builtin_legends
for legend in legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


In [13]:
Map.add_legend(builtin_legend='NLCD')
Map

Map(center=[38, -95], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tra…

### 4.6.2 カスタム凡例の作成

In [14]:
Map = geemap.Map()

# 凡例辞書はそのままで OK
legend_dict = {
    '11 Open Water': '466b9f',
    '12 Perennial Ice/Snow': 'd1def8',
    '21 Developed, Open Space': 'dec5c5',
    '22 Developed, Low Intensity': 'd99282',
    '23 Developed, Medium Intensity': 'eb0000',
    '24 Developed High Intensity': 'ab0000',
    '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
    '41 Deciduous Forest': '68ab5f',
    '42 Evergreen Forest': '1c5f2c',
    '43 Mixed Forest': 'b5c58f',
    '51 Dwarf Scrub': 'af963c',
    '52 Shrub/Scrub': 'ccb879',
    '71 Grassland/Herbaceous': 'dfdfc2',
    '72 Sedge/Herbaceous': 'd1d182',
    '73 Lichens': 'a3cc51',
    '74 Moss': '82ba9e',
    '81 Pasture/Hay': 'dcd939',
    '82 Cultivated Crops': 'ab6c28',
    '90 Woody Wetlands': 'b8d9eb',
    '95 Emergent Herbaceous Wetlands': '6c9fb8'
}

# 正しく ImageCollection から NLCD2019 の画像を選択
nlcd_collection = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
nlcd_2019 = nlcd_collection.filter(ee.Filter.eq('system:index', '2019')).first()

# landcover バンドを選択
landcover = nlcd_2019.select('landcover')

# マップに追加
Map.addLayer(landcover, {}, 'NLCD 2019 Land Cover')

# 凡例追加
Map.add_legend(legend_title="NLCD Land Cover Classification", legend_dict=legend_dict)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

## 4.7 Leafmap：Google Earth Engineの代替

In [15]:
from ipyleaflet import *


m = Map(center=[48.8566, 2.3522], zoom=10, height=600, widescreen=False, basemap=basemaps.Esri.WorldStreetMap)
m

Map(center=[48.8566, 2.3522], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [16]:
m.interact(zoom=(5, 10, 1))

Box(children=(IntSlider(value=10, description='zoom', max=10, min=5),))

In [18]:
minimap = Map(
    zoom_control=False,
    attribution_control=False,
    zoom=5,
    center=m.center,
    basemap=basemaps.Esri.WorldStreetMap
)
minimap.layout.width = '200px'
minimap.layout.height = '200px'
link((minimap, 'center'), (m, 'center'))
minimap_control = WidgetControl(widget=minimap, position='bottomleft')
m.add_control(minimap_control)
m

Map(bottom=90385.0, center=[48.8566, 2.3522], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…